In [1]:
"""
Generate the full spectral response for both DC and Denver
Author: maxwell.cook@colorado.edu
"""

import os, sys, gc, time
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import numpy as np
import pyproj
import earthpy.plot as ep
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Set some matplotlib settings
%matplotlib inline
plt.rcParams['font.family'] = 'Times New Roman'

# environment
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/'
projdir = os.path.join(maindir,'data/spatial/')
proj = 32618  # UTM Zone 18N

print("Ready!")

Ready!


In [2]:
# load the reference data
dc_ref_fp = os.path.join(projdir,'mod/dc_data/training/dc_data_reference_footprints.gpkg')
denver_ref_fp = os.path.join(projdir,'mod/denver_data/training/denver_data_reference_footprints.gpkg')
# load the image data
dc_psb_fp = os.path.join(projdir,'mod/dc_data/planet-data/dc_0623_psscene8b_stack.tif')
denver_psb_fp = os.path.join(projdir,'mod/denver_data/planet-data/denver_0815_psscene8b_stack.tif')

In [3]:
# calculate the spectral response data frame
out_fp = os.path.join(maindir,'data/tabular/mod/denver_data/denver_spectral_response.csv')

if not os.path.exists(out_fp):
    t0 = time.time()
    
    # Create the band stats class
    band_stats_obj = BandStatistics(denver_ref_fp, denver_psb_fp, unique_id='uid')
    # Run parallel band statistics computation
    sampled = band_stats_obj.parallel_compute_stats(stat='mean')

    # save the file out
    sampled.to_csv(out_fp)
    print(f"Saved to: {out_fp}")

    t1 = (time.time() - t0) / 60
    print(f"Total elapsed time: {t1:.2f} minutes.")
    print("\n~~~~~~~~~~\n")

else:
    print("Loading sampled data.")
    denver_sampled = pd.read_csv(out_fp)

Loading sampled data.


In [4]:
denver_sampled.head()

,Unnamed: 0,uid,coastal_blue,blue,green_i,green,yellow,red,rededge,nir,NDRE,VgNIRBI,VrNIRBI,NDBIbg,NDBIrg,NISI,mnf1
0,0,78TL,445.666667,590.444444,740.000000,835.777778,859.555556,898.888889,1107.222222,2211.444444,0.334336,-0.454585,-0.424544,-0.172487,0.038642,0.016709,-1.667098
1,1,269TL,222.457627,333.372881,495.627119,602.067797,664.542373,700.966102,890.677966,2044.830508,0.394653,-0.545364,-0.492504,-0.302047,0.069057,-0.122478,-16.492368
2,2,490TL,360.100000,478.400000,642.666667,798.700000,1028.666667,1143.500000,1384.600000,2537.633333,0.296772,-0.522985,-0.383330,-0.256231,0.173341,-0.031092,-4.582541
3,3,497TL,154.111111,256.444444,445.666667,509.000000,499.000000,488.555556,743.222222,2290.777778,0.508524,-0.635290,-0.646997,-0.331815,-0.021478,-0.291740,-21.659212
4,4,537TL,548.833333,727.333333,967.166667,1130.666667,1309.833333,1344.500000,1548.833333,2506.500000,0.235801,-0.377683,-0.301264,-0.217550,0.086520,0.122086,13.339019


In [8]:
# calculate the spectral response data frame
out_fp = os.path.join(maindir,'data/tabular/mod/dc_data/dc_spectral_response.csv')

if not os.path.exists(out_fp):
    t0 = time.time() 
    
    # Create the band stats class
    band_stats_obj = BandStatistics(dc_ref_fp, dc_psb_fp, unique_id='uid')
    # Run parallel band statistics computation
    dc_sampled = band_stats_obj.parallel_compute_stats(stat='mean')

    # save the file out
    dc_sampled.to_csv(out_fp)
    print(f"Saved to: {out_fp}")
    
    t1 = (time.time() - t0) / 60
    print(f"Total elapsed time: {t1:.2f} minutes.")
    print("\n~~~~~~~~~~\n")

else:
    print("Loading sampled data.")
    dc_sampled = pd.read_csv(out_fp)

Loading sampled data.


In [6]:
# save the file out
dc_sampled.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/data/tabular/mod/dc_data/dc_spectral_response.csv


In [7]:
dc_sampled.head()

,uid,coastal_blue,blue,green_i,green,yellow,red,rededge,nir,NDRE,VgNIRBI,VrNIRBI,NDBIbg,NDBIrg,NISI,mnf1
0,1CS,675.833333,658.250000,818.583333,938.750000,1022.750000,857.666667,1306.166667,3016.083333,0.395654,-0.524777,-0.556995,-0.174235,-0.047747,-0.106653,1.782163
1,3CS,837.368421,848.315789,1015.631579,1150.210526,1314.578947,1191.526316,1440.368421,2413.421053,0.253496,-0.355253,-0.340484,-0.151595,0.016156,0.135617,12.189694
2,9CS,742.111111,745.259259,863.962963,981.333333,1078.814815,958.925926,1241.481481,2405.481481,0.318952,-0.420815,-0.430460,-0.135753,-0.011701,0.054640,4.714900
3,19CS,560.933333,641.933333,800.800000,922.666667,1050.533333,914.933333,1191.800000,2532.066667,0.359351,-0.466395,-0.470459,-0.180235,-0.006337,-0.014110,1.564940
4,27CS,772.222222,707.444444,869.666667,980.444444,1076.555556,977.777778,1273.333333,2845.555556,0.381331,-0.487240,-0.488584,-0.161741,-0.001919,-0.032534,4.016014


In [ ]:
gc.collect()